## 将txt文件转换为csv文件

In [ ]:
import numpy as np
import pandas as pd
import csv

In [2]:
reader = open('./datasets/char_embed.txt')
data1 = reader.readlines()
with open("./datasets/char_embed.csv","w")as csvfile:
    for row in data1:
        row = row.split()
        writer = csv.writer(csvfile)
        writer.writerow(row)
        
reader = open('./datasets/word_embed.txt')
data2 = reader.readlines()
with open("./datasets/word_embed.csv","w")as csvfile:
    for row in data2:
        row = row.split()
        writer = csv.writer(csvfile)
        writer.writerow(row)

## 读取csv文件，查看内容

In [3]:
train_master = pd.read_csv('./datasets/train.csv',encoding='gbk')
test_master = pd.read_csv('./datasets/test.csv',encoding='gbk')
question = pd.read_csv('./datasets/question.csv',encoding='gbk')
char_embed = pd.read_csv('./datasets/char_embed.csv',encoding='gbk')
word_embed = pd.read_csv('./datasets/word_embed.csv',encoding='gbk')

In [4]:
train_master.head(5)

,label,q1,q2
0,1,Q397345,Q538594
1,0,Q193805,Q699273
2,0,Q085471,Q676160
3,0,Q189314,Q438123
4,0,Q267714,Q290126


In [5]:
test_master.head(5)

,q1,q2
0,Q017571,Q006012
1,Q728241,Q542572
2,Q166997,Q118270
3,Q422420,Q514386
4,Q354329,Q032485


In [6]:
question.head(5)

,qid,words,chars
0,Q000000,W05733 W05284 W09158 W14968 W07863,L1128 L1861 L2218 L1796 L1055 L0847 L2927
1,Q000001,W17378 W17534 W03249 W01490 W18802,L2214 L1980 L0156 L1554 L2218 L1861 L3019 L010...
2,Q000002,W17378 W08158 W20171 W11246 W14759,L2214 L2350 L2568 L1969 L2168 L0694 L3012 L256...
3,Q000003,W11385 W14103 W02556 W13157 W09749,L0762 L2172 L1971 L2034 L2039 L2271 L1346 L223...
4,Q000004,W17508 W18238 W02952 W18103,L0018 L2321 L1346 L3019 L0104 L0902 L0354 L0362


In [7]:
char_embed.head(5)

,L0000,-0.54645991,2.28509140,-3.08430910,1.06466067,-2.09087992,0.65149629,-2.42987657,-2.26238489,-1.98188436,...,0.20739675,1.47637320,0.86374396,-0.34182572,0.43323368,-0.73032355,0.21595474,-0.52845210,-0.34052843,-2.01874685
0,L0001,-9.016356,-3.801084,-7.210567,3.052900,-1.340958,1.395385,-5.482922,-7.407759,-3.611857,...,-0.818142,4.968217,-4.254042,-0.709047,1.288105,-1.222849,-5.521402,-2.653049,1.868731,2.147064
1,L0002,-0.138824,0.219148,-0.890053,0.106301,-0.494364,0.550745,-0.279467,-0.048898,-0.021813,...,0.247383,0.576698,1.261507,0.446992,-0.418965,-0.278471,1.426156,-0.579678,-0.322354,-0.661302
2,L0003,-0.265548,0.465279,-0.611959,0.356313,-0.533891,0.196803,-0.376730,-0.316881,-0.516114,...,0.136175,0.952863,0.902963,-0.630882,0.153895,-0.260504,0.665082,0.126045,-0.281423,-0.155615
3,L0004,0.099463,-3.760915,0.462665,-3.754872,11.792088,7.473687,-3.999821,-0.934766,1.401825,...,7.900008,0.938448,9.856948,2.781040,-3.262262,-2.092505,3.462315,-0.904220,1.935117,3.409496
4,L0005,3.059664,-1.570498,-0.629994,1.990173,0.433813,-0.962803,4.283813,4.137911,4.075794,...,5.185996,1.077251,-5.744498,-1.455833,-2.667485,-3.010494,-0.601640,0.117071,-1.554845,-4.167559


In [8]:
word_embed.head(5)

,W00000,0.16931550,-0.06389757,0.11528583,-0.07767122,0.06718445,0.01933860,0.03959615,-0.02622881,-0.16007800,...,0.06115149,0.04451883,-0.19482729,0.12245613,0.12278509,-0.15415290,-0.11657750,-0.12778564,0.11059313,-0.17108430
0,W00001,1.548212,-1.052776,1.192632,0.760363,1.594398,1.478917,-1.555349,0.401968,1.588316,...,-1.898932,0.129864,-2.062325,0.068316,0.540282,-1.682620,-0.816290,-1.464458,-0.361792,0.943322
1,W00002,0.934084,0.106135,-0.391749,-0.209661,-0.558696,-0.942362,-0.274353,-0.232077,-1.024267,...,-0.357264,-0.451105,-0.724659,0.525233,0.290343,0.357838,-0.042750,1.315442,-0.167775,-0.393665
2,W00003,-1.407966,0.034697,-2.333980,1.991531,-0.073803,-0.966899,-0.926573,-2.161519,0.961513,...,0.547399,1.300992,1.893234,0.833304,-0.531153,1.249110,-0.959852,2.486905,0.925652,-0.676427
3,W00004,-0.213362,0.478031,0.033140,-0.096893,0.077964,-0.149793,-0.163284,0.470706,0.138422,...,-0.185111,-0.171314,0.166296,0.291282,-0.202618,0.192025,-0.145072,0.151652,0.353130,-0.073013
4,W00005,0.154526,-1.702239,1.824775,-2.003592,-0.022180,-1.423040,-1.004813,-0.649550,1.499301,...,0.498754,-0.740963,-0.108896,-2.453417,-0.602805,0.039574,-0.245428,0.215308,1.388125,1.267395


## 数据清洗

In [13]:
train_sum = train_master.isnull().sum()
print(train_sum)
test_sum = test_master.isnull().sum()
print(test_sum)
question_sum = question.isnull().sum()
print(question_sum)

label    0
q1       0
q2       0
dtype: int64
q1    0
q2    0
dtype: int64
qid      0
words    0
chars    0
dtype: int64


In [17]:
train_master.shape

(254386, 3)

In [18]:
train_master.index

RangeIndex(start=0, stop=254386, step=1)

In [57]:
train = pd.DataFrame(train_master)
question_list1 = pd.DataFrame(train.iloc[:,1:2])
question_list2 = pd.DataFrame(train.iloc[:,2:3])
question_list = pd.concat([question_list1,question_list2], sort = True)
question_list = question_list.drop(columns = ['q2'])

In [59]:
question_list.shape

(508772, 1)

In [60]:
question_list.groupby(['q1'])

In [61]:
question_list.head(10)

,q1
0,Q397345
1,Q193805
2,Q085471
3,Q189314
4,Q267714
5,Q369715
6,Q537991
7,Q255847
8,Q639518
9,Q521550


In [62]:
question_list.shape

(508772, 1)